### Creación de db, de tabla e insertar lo dicho por la API de cohere

Creación de base de datos

In [53]:
import pymysql
import os
from dotenv import load_dotenv
import cohere


load_dotenv();

In [72]:
username = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT") 

In [73]:
db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

In [59]:
cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.40'}


In [19]:
create_db = '''CREATE DATABASE db_history'''
cursor.execute(create_db)

1

In [74]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'db_history'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

In [75]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE db_history'''
cursor.execute(use_db)

0

In [35]:
# crear una tabla que se llame interviews

create_table = '''
CREATE TABLE interviews (
    id INT AUTO_INCREMENT,
    session_id VARCHAR(255) NOT NULL,
    tema VARCHAR(255) NOT NULL,
    preguntas TEXT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    primary key (id)
)
'''
cursor.execute(create_table)

0

In [47]:
# crear una tabla que se llame interviews

create_table = '''
CREATE TABLE evaluations (
    id INT AUTO_INCREMENT,
    session_id VARCHAR(255) NOT NULL,
    respuestas TEXT NOT NULL,
    evaluaciones TEXT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    primary key (id)
)
'''
cursor.execute(create_table)

0

In [62]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_db_history': 'evaluations'},
 {'Tables_in_db_history': 'interviews'}]

***Pasamos a insertar los datos sacados por la API***

In [54]:
co = cohere.ClientV2("O7Cjn2pyi02rLIJE9azktO8hRehHboQjeA1HM9fQ")


In [55]:
prompt='Un entrevistador ha realizado la siguiente pregunta: ¿Puede describir un caso en el que haya utilizado técnicas de aprendizaje automático para resolver un problema del mundo real y cuáles fueron los resultados?. Ante esa pregunta yo he respondido esto:para predecir un fallo de una máquina. Evaluame cómo lo he hecho, diciéndome los errores. Gracias'

In [ ]:
response = co.chat(
    model="command-r-plus",
    messages=[{"role": "user", "content": "Genera 10 preguntas de entrevista sobre data science"}]
)
preguntas = [msg.text for msg in response.message.content]

In [56]:
response = co.chat(
    model="command-r-plus", 
    messages=[{"role": "user", "content": prompt}])

print(prompt)
print(response.message.content[0].text)

Un entrevistador ha realizado la siguiente pregunta: ¿Puede describir un caso en el que haya utilizado técnicas de aprendizaje automático para resolver un problema del mundo real y cuáles fueron los resultados?. Ante esa pregunta yo he respondido esto:para predecir un fallo de una máquina. Evaluame cómo lo he hecho, diciéndome los errores. Gracias
Su respuesta es un poco breve y vaga. Aquí hay algunos comentarios y sugerencias para mejorarla:

Contexto: Proporcione más contexto sobre el problema que estaba tratando de resolver. ¿Qué tipo de máquina era y en qué contexto se utilizaba? ¿Por qué era importante predecir su falla?

Detalles técnicos: Explique brevemente las técnicas de aprendizaje automático específicas que utilizó para abordar este problema. Por ejemplo, ¿utilizó modelos de clasificación, regresión o series de tiempo? ¿Cómo preparó y procesó los datos para el modelo?

Resultados: Describa los resultados con más detalle. ¿Cuán preciso fue su modelo en la predicción de falla

In [21]:
insert_data = '''INSERT INTO history (prompt, answer) VALUES (%s, %s)'''
cursor.execute(insert_data, (prompt, response.message.content[0].text))


1

In [77]:
sql = '''SELECT * FROM evaluations'''
cursor.execute(sql)
mi_lista = cursor.fetchall()
print(mi_lista)

[{'id': 1, 'session_id': 'aeadb9a6-0316-4fb2-851d-aee408de263e', 'respuestas': 'Haciendo un Eda. \nUso la lógica.\nEn el máster en un trabajo.', 'evaluaciones': '¡Seguro! Aquí hay una breve evaluación de tus respuestas:\n\n1. "Haciendo un Eda": Parece que estás sugiriendo que realizas un "Análisis Exploratorio de Datos" (EDA) para abordar problemas complejos de análisis de datos. Aunque el EDA es una técnica valiosa, puede ser útil proporcionar más detalles sobre tu proceso. Por ejemplo, ¿cómo decides qué técnicas analíticas utilizar? ¿Cóes una estructura o un marco de trabajo específico que sigues? Proporcionar un ejemplo o dos de cómo has aplicado con éxito el EDA en el pasado podría ayudar a fortalecer tu respuesta.\n\n2. "Uso la lógica": Esta respuesta es concisa, pero puede ser beneficioso proporcionar un poco más de contexto. Podrías mencionar técnicas o enfoques específicos que utilices para garantizar la precisión, como la validación cruzada, el control de calidad de datos o el

In [64]:
# Borrar la data de la tabla history
cursor.execute('TRUNCATE TABLE interviews')
cursor.execute('TRUNCATE TABLE evaluations')
cursor.fetchall()

[]

***Guardar cambios***

In [8]:
db.commit()

***Cerrar db***

In [78]:
# Cerrar conexión mysql
db.close()